In [6]:
import pandas as pd

df = pd.read_csv('train_June.csv')
df.head()

,prompt,completion
0,What is the role of a Business admin in BluH ?,Business admin is responsible for the all admi...
1,What is the role of a Hatchery owner in BluH?,Hatchery owner is responsible for the all acti...
2,What is the role of a Hatchery manager in BluH?,Hatchery manger is responsible for the all act...
3,What is the role of an Accountant in BluH?,Accountant is responsible for the all MTL (mat...
4,What is the role of a Maturation Technician in...,Maturation technician is responsible for the a...


In [7]:
import pinecone
import pandas as pd
from pinecone import Pinecone, ServerlessSpec

# Pinecone API key and environment
PINECONE_API_KEY = 'pcsk_3baywT_NwSZhYb19aoV3RX8w8n84jMhC4PmW8VWTBba9SYD4soUoS1oFFCzkEoPsvB5Ra'
PINECONE_ENV = 'us-east-1'  # e.g., 'us-west1-gcp'

# Initialize Pinecone
pc = Pinecone(
    api_key=PINECONE_API_KEY
)

# Create a new index in Pinecone (run this once)
index_name = "domain-knowledge"
if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name=index_name, 
            dimension=1024, 
            metric='cosine',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-east-1'
            )
        )


/home/eebee/anaconda3/envs/bluH/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '021851a89c4e0b55f4fa440f74310d40', 'Date': 'Tue, 26 Nov 2024 10:20:20 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [8]:
from sentence_transformers import SentenceTransformer

# Connect to the index
index = pinecone.Index(api_key=PINECONE_API_KEY, host='https://domain-knowledge-xq431jt.svc.aped-4627-b74a.pinecone.io')

# Load the CSV file
data = pd.read_csv('train_June.csv')

# Load the SentenceTransformer model
model = SentenceTransformer('thenlper/gte-large')

# Generate embeddings and upsert into Pinecone
for idx, row in data.iterrows():
    prompt = row['prompt']
    completion = row['completion']
    
    # Generate embedding for the prompt
    embedding = model.encode(prompt).tolist()
    
    # Upsert into Pinecone with a unique ID
    index.upsert([(str(idx), embedding, {"completion": completion})])

print("Embedding and upserting completed.")

Embedding and upserting completed.


In [ ]:
def search_pinecone(query, top_k=3):
    # Generate embedding for the user query
    query_embedding = model.encode(query).tolist()
    
    # Use keyword arguments for the query method
    query_results = index.query(
        vector=query_embedding,  # Use `vector` instead of a positional argument
        top_k=top_k,  # Specify the number of top results
        include_metadata=True  # Include metadata for the results
    )
    
    # Extract the closest completions and their metadata
    completions = [result['metadata']['completion'] for result in query_results['matches']]
    return completions

from groq import Groq

# Initialize the Groq client
client = Groq(api_key="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")

def generate_answer(query, retrieved_completions, chat_history):
    # Combine the user query and retrieved completions into a prompt
    system_message = {
        "role": "system",
        "content": (
        "You are a knowledgeable and helpful assistant for shrimp hatcheries. "
        "Your role is to provide accurate and concise information related to shrimp farming, "
        "hatchery operations, disease management, water quality monitoring, feed optimization, and other relevant topics. "
        "Always base your responses on the provided context, and if specific context is not available, offer general best practices."
        )
    }
    context_messages = [
        {"role": msg["role"], "content": msg["content"]} for msg in chat_history[-5:]
    ]

    user_message = {
        "role": "user",
        "content": f"User query: {query}\n\nRelevant information:\n" + "\n".join(retrieved_completions)
    }
    
    all_messages = [system_message] + context_messages + [user_message]

    # Generate response with Groq API
    chat_completion = client.chat.completions.create(
        messages=all_messages,
        model="llama3-8b-8192",
        temperature=0.5,
        max_tokens=1024,
        top_p=1,
        stop=None,
        stream=False,  # Stream partial results
    )
    
    # Aggregate and return the full response
    response_text = chat_completion.choices[0].message.content
    
    return response_text


def handle_user_query(query, chat_history):
    # Step 1: Retrieve closest completions from Pinecone
    closest_completions = search_pinecone(query)
    
    # Step 2: Pass query and retrieved completions to the LLM for final answer
    final_answer = generate_answer(query, closest_completions, chat_history)
    
    return final_answer

In [21]:
user_query = "What is the role of a Business admin in BluH ?"
response = handle_user_query(user_query)
print("Bot response:", response)

Bot response: Based on the provided context, it appears that the Business Admin in BluH has a crucial role in managing the administrative activities of the hatchery and farm businesses. Their responsibilities include:

1. Infrastructure creation: Setting up and configuring the necessary infrastructure for the hatchery and farm businesses.
2. Inventory item creation: Creating and managing inventory items, such as equipment, supplies, and materials, for the hatchery and farm businesses.
3. Role creation and assignments: Creating and assigning roles to users within the hatchery and farm businesses, ensuring that each user has the necessary permissions and access to perform their duties.

In essence, the Business Admin is responsible for setting up and maintaining the administrative backbone of the hatchery and farm businesses, allowing other teams to focus on their core responsibilities, such as shrimp farming, disease management, and water quality monitoring.


In [22]:
import os
from dotenv import load_dotenv

load_dotenv()


PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENV = os.getenv("PPINECONE_ENV")
PINECONE_HOST = os.getenv("PINECONE_HOST")

In [25]:
PINECONE_ENV